# Sistemas de Recomendación con factorización de matrices

Com vimos, el objetivo de los algoritmos de recomendación basados en filtrado colaborativo es sugerir nuevos productos o predecir la utilidad de un producto para un usuario, en función del comportamiento anterior de dicho usuario y las reviews o ratings de otros usuarios similares. Sin embargo, estos sistemas tienen algunos problemas como la falta de datos y la escalabilidad. En este contexto, existe una preocupación teórica respecto a los enfoques basados en datos sin procesar como los ratings.

El problema clave es que las matrices de ratings pueden ser presentar representaciones "ruidosas" de los gustos y preferencias de los usuarios. Al utilizar enfoques de "vecindad" basados en la distancia en datos sin procesar, se busca la coincidencia entre los detalles escasos y de bajo nivel que se supone que representan los vectores de preferencia de los usuarios, en lugar de buscar la coincidencia con los propios vectores. Es una diferencia sutil, pero importante.

Por ejemplo, si un usuario escuchó diez canciones de los *Backstreet Boys* y otro usuario ha escuchado diez canciones diferentes de los *Backstreet Boys*, la matriz de acciones de los usuarios no se solaparán y la semejanza entre ambos vectores sería 0, aún cuando sea probable que ambos usuarios compartan algunas preferencias.

El uso de las características de los elementos (como el género) podría ayudar a solucionar este problema, pero no completamente. Por ejemplo, qué sucedería si a ambos usuarios les gustan las canciones con una "gran narración" independientemente de su género? Para resolver esto es necesario contar con métodos que permitan derivar gustos y vectores de preferencias a partir de los datos sin procesar. Para ello, vamos a recurrir a la *factorización de matrices*.



Una forma de manejar el problema de escalabilidad y falta de datos creado por el filtrado colaborativo es aprovechar un modelo de factores latentes para capturar la similitud entre usuarios y elementos. Esencialmente, queremos convertir el problema de recomendación en un problema de optimización. Para esto vamos a utilizar *Singular Value Decomposition* (SVD). El objetivo de SVD es descomponer una matriz en una aproximación de menor rango.  

<p align="center">
<img src="https://hackernoon.com/hn-images/1*haUDjEiQmG0RapR0SHos6Q.png" width="600">
</p>

En la fórmula, 
* $X$ denota la matriz de utilidad (nuestra matriz de ratings).
* $U$ es una matriz singular izquierda, que representa la relación entre los usuarios y los factores latentes. En otras palabras, cuál es el interés de un usuario por un elemento.
* $S$ es una matriz diagonal que describe la fuerza de cada factor latente.
* $V^T$ es una matriz singular derecha, que indica la similitud entre los elementos y los factores latentes. En otras palabras, cuán relevante es una característica para los elementos.

*Qué son los "factores latentes?"*
Es una idea amplia que describe una propiedad o concepto que tiene un usuario o un elemento. Por ejemplo, para la música, el factor latente puede referirse al género al que pertenece la música. SVD disminuye la dimensión de la matriz de utilidad extrayendo sus factores latentes. Esencialmente, asignamos a cada usuario y cada elemento en un espacio latente con dimensión $r$. Por lo tanto, ayuda a comprender mejor la relación entre usuarios y elementos a medida que se vuelven directamente comparables. La siguiente figura ilustra esta idea.

<p align="center">
<img src="https://hackernoon.com/hn-images/1*GUw90kG2ltTd2k_iv3Vo0Q.png
" width="600">
</p>

Una vez que tenemos la descomposición, para obtener la aproximación de menor rango, nos vamos a quedar solo con las mejores $k$ carcterísticas que son las más importantes para representar a los vectores de preferencias.





Si bien, SVD permite solucionar los problemas de escalabilidad y escasez que plantea el filtrado colaborativo con éxito, SVD no está exenta de defectos. El principal inconveniente de SVD es que no da explicaciones del porqué de las recomendaciones a los usuarios.


Nuevamente vamos a usar un [dataset](https://raw.githubusercontent.com/tommantonela/sistemasRecomendacion2019/master/User-User%20Collaborative%20Filtering%20-%20movie-row.csv) de películas. 
Este dataset tiene los ratings que $25$ usuarios $u$ le asignaron a $100$ películas $m$. Si el usuario $u$ no le asignó rating a la película $m$, la celda correspondiente está vacía. 

In [0]:
import pandas as pd 
import numpy as np

url = 'https://raw.githubusercontent.com/tommantonela/sistemasRecomendacion2019/master/User-User%20Collaborative%20Filtering%20-%20movie-row.csv'
df = pd.read_csv(url, sep=',',index_col=0)

print(str(df.shape))
df.head()


Similar a como habíamos hecho antes, vamos a restar la media de los ratings para hacer una "normalización". 

In [0]:
normalised_mat = df - np.asarray([(np.mean(df, 1))]).T

normalised_mat.head()

Ahora ya podemos aplicar SVD. Hay dos opciones, o lo implementamos de cero nosotros; o podemos usar los métodos que nos provee numpy (spoiler alert: usar numpy es más fácil!).

In [0]:
A = normalised_mat.T / np.sqrt(df.shape[0] - 1)

print(A)

U, S, V = np.linalg.svd(A)

print(V.tranpose())

Qué pasó? El SVD no converge. Esto es por la presencia de los $NaN$ en la matriz de ratings. Esos $NaN$ nos indican los ratings faltantes de los usuarios, es decir, los ratings que queremos predecir.

Qué tenemos que hacer? Darles un valor inicial. Supongamos que decidimos darle un valor de $0$.


In [0]:
normalised_mat = normalised_mat.replace(np.nan, 0)

normalised_mat.head()

In [0]:
A = normalised_mat.T / np.sqrt(df.shape[0] - 1)
print(A.shape)
U, S, V = np.linalg.svd(A)

print(V.T)

Una vez que tenemos la descomposición, lo que vamos a hacer es quedarnos con las $k$ características y encontrar las películas más similares a una dada.

In [0]:
def top_cosine_similarity(data, movie_id, top_n=10):
    movie_row = data[movie_id, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(movie_row, data.T) / (magnitude[movie_id] * magnitude)
    
    sort_indexes = np.argsort(-similarity)
    
    return sort_indexes[:top_n]

In [0]:
k = 10
movie_id = 1 # 12: Finding Nemo (2003)

sliced = V.T[:, :k] # representative data con esto estamos seleccionado las k features que "mejor" describen a nuestras películas

movies = top_cosine_similarity(sliced,movie_id)

print(df.index.values[movies]) # convertimos los índices a los títulos

In [0]:
Sp = S[:k]
Vp = V[:k, :]
Sdp = np.zeros((U.shape[0],Vp.shape[0]))
Sdp[range(Sp.shape[0]), range(Sp.shape[0])] = Sp

A_reconstruida = U.dot(Sdp).dot(Vp)
print(A_reconstruida)

print(np.allclose(A,A_reconstruida))

Ahora bien, parecen realmente similares las películas que encontramos? 

Recordemos que comenzamos el SVD asignando valores de $0$ a todos los ratings que no conocíamos. Podríamos también inicializar con valores random, pero qué nos garantiza que esos random nos ayuden a encontrar las mejores aproximaciones de los ratings?

Lo que se puede hacer es encontrar una aproximación de la matriz SVD con valores $NaN$ usando un procedimiento iterativo:

1. Completar los $NaN$ con una aproximación. Por ejemplo, reemplazarlos por los promedios de las columnas.
2. Realizar SVD sobre esta matriz.
3. Reconstruir la matriz resultado del SVD para obtener una mejor aproximación de los valores faltantes.
4. Repetir los pasos 2 y 3 hasta convergencia.

Esto es una forma del algoritmo de Expectation Maximization (EM), donde el paso E actualizada las estimaciones de los valores faltantes del SVD y M calcula el SVD sobre los valores estimados.

Ver ["Methods for large scale SVD with missing values"](https://www.cs.uic.edu/~liub/KDD-cup-2007/proceedings/missing-value-Kurucz.pdf) para una explicación teórica más detallada.

In [0]:
from scipy.sparse.linalg import svds
from functools import partial


def emsvd(Y, k=None, tol=1E-3, maxiter=None,column=1):
    """
    Approximate SVD on data with missing values via expectation-maximization

    Inputs:
    -----------
    Y:          (nobs, ndim) data matrix, missing values denoted by NaN/Inf
    k:          number of singular values/vectors to find (default: k=ndim)
    tol:        convergence tolerance on change in trace norm
    maxiter:    maximum number of EM steps to perform (default: no limit)

    Returns:
    -----------
    Y_hat:      (nobs, ndim) reconstructed data matrix
    mu_hat:     (ndim,) estimated column means for reconstructed data
    U, s, Vt:   singular values and vectors (see np.linalg.svd and 
                scipy.sparse.linalg.svds for details)
    """

    if k is None:
        svdmethod = partial(np.linalg.svd, full_matrices=False)
    else:
        svdmethod = partial(svds, k=k)
    if maxiter is None:
        maxiter = np.inf

    # initialize the missing values to their respective column means
    mu_hat = np.nanmean(Y, axis=0, keepdims=1)
    valid = np.isfinite(Y)
    Y_hat = np.where(valid, Y, mu_hat)

    halt = False
    ii = 1
    v_prev = 0

    while not halt:

        # SVD on filled-in data
        U, s, Vt = svdmethod(Y_hat - mu_hat)

        # impute missing values
        Y_hat[~valid] = (U.dot(np.diag(s)).dot(Vt) + mu_hat)[~valid]

        print(str(ii)," :: ",*Y_hat.transpose()[column])

        # update bias parameter
        mu_hat = Y_hat.mean(axis=0, keepdims=1)

        # test convergence using relative change in trace norm
        v = s.sum()
        if ii >= maxiter or ((v - v_prev) / v_prev) < tol:
            halt = True
        ii += 1
        v_prev = v

    return Y_hat, mu_hat, U, s, Vt

In [0]:
Y_hat, mu_hat, U, s, Vt = emsvd(df,1) # simplemente lo invocamos y hace todo solito!

print("Termino!")
print(Y_hat.shape)

import sys
np.set_printoptions(threshold=sys.maxsize)
print(Y_hat.transpose())


#### Tarea!
Dada `Y_hat` que es la matriz de ratings reconstruida, encontrar las películas más similares a Nemo.


In [0]:
# TODO!


#### Tarea 2!
Dada `Y_hat` comparar las diferencias entre los ratings originales y los estimados para un usuario y película dado.

In [0]:
# TODO 2! usuario


In [0]:
# TODO 2! película



## Y ahora con redes neuronales!

Definimos algunas funciones de ayuda útiles

In [0]:
def mapping(serie):
    data = list(set(serie))
    data.sort()
    return {k:i for i, k in enumerate(data)}
    
def to_matrix(ds):
    users = ds['user_id'].tolist()
    movies = ds['movie_title'].tolist()
    ratings = ds['rating'].tolist()
    users_mapping = mapping(users)
    movies_mapping = mapping(movies)
    mat = np.empty((len(users_mapping), len(movies_mapping)))
    mat[:,:] = float('nan')
    for u, m, r in zip(users, movies, ratings):
        mat[users_mapping[u], movies_mapping[m]] = r
    return mat, users_mapping, movies_mapping

def to_lists(ds, users_mapping, movies_mapping):
    users = ds['user_id'].tolist()
    movies = ds['movie_title'].tolist()
    ratings = ds['rating'].tolist()
    x_u = []
    x_m = []
    y_r = []
    for u, m, r in zip(users, movies, ratings):
        if u not in users_mapping or m not in movies_mapping:
            continue
        x_u.append(users_mapping[u])
        x_m.append(movies_mapping[m])
        y_r.append(r)
    return [np.asarray(x_u), np.asarray(x_m)], np.asarray(y_r)

In [0]:
# cargamos el dataset
url = 'https://raw.githubusercontent.com/tommantonela/sistemasRecomendacion2019/master/ml-100k/u.data'
df = pd.read_csv(url, sep='\t', names=['user_id','movie_id','rating','timestamp'])

url = 'https://raw.githubusercontent.com/tommantonela/sistemasRecomendacion2019/master/ml-100k/u.item'
movie_info = pd.read_csv(url,sep='|', encoding='latin-1', header=None, names=['movie_id','movie_title','release_date','movie_release_date',
                                                                              'IMDb url','unknown','Action','Adventure','Animation','Children','Comedy',
                                                                              'Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical',
                                                                              'Mystery','Romance','Sci-Fi','Thriller','War','Western'])

movie_info = movie_info.drop('movie_release_date', axis=1) # eliminamos la columna movie_release_date que es NaN para todos los registros
movie_info = movie_info.drop('IMDb url', axis=1) # eliminamos esta columna que no aporta ninguna información relevante

df = pd.merge(df, movie_info, on='movie_id')

mat, users_mapping, movies_mapping= to_matrix(df)

Y ahora si, el modelo

In [0]:
from keras.layers import Input, Dense, Embedding, concatenate, Dropout, Flatten
from keras.models import Model

size = 100 # define la cantidad de features que uso para representar las pelis y los usuarios

user = Input((1,))
embedding_user = Embedding(len(users_mapping)+1, size)(user)

movie = Input((1,))
embedding_movie = Embedding(len(movies_mapping)+1, size)(movie)

# aproximador universal - perceptron multicapa
concat = concatenate([embedding_user, embedding_movie], axis=-1)
concat = Flatten()(concat)
# si sacamos estas dos (dense y dropout) es un regresor lineal
dense = Dense(size)(concat)
dropout = Dropout(0.2)(dense)
predict = Dense(1)(dropout)

model = Model([user, movie], predict)
model.compile(loss='mae', optimizer='adam')
model.summary()

In [0]:
X_train, y_train = to_lists(df, users_mapping, movies_mapping)

In [0]:
model.fit(X_train, y_train, epochs=3)

In [0]:
model.predict(X_train)

**Tarea!** Dadas las predicciones, comparar las diferencias entre los ratings originales y los estimados para un usuario dado.

In [0]:
# TODO!